# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [1]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
browser = Browser('chrome')

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
html = browser.html
soup = soup(html, 'html.parser')

In [ ]:
# Extract all rows of data
mars_temp_df = pd.read_html('https://static.bc-edx.com/data/web/mars_facts/temperature.html')
mars_temp_df

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
# Create an empty list
table = soup.find('table')
rows = []
# Loop through the scraped data to create a list of rows
for row in table.find_all('tr'):
    cells = row.find_all(['th', 'td'])
    rows.append([cell.get_text(strip=True) for cell in cells])

# Variables to store the table headers and the actual data
headers = rows[0]
data_rows = rows[1:]

In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
mars_temp_df = pd.DataFrame(data_rows, columns=headers)

In [ ]:
# Confirm DataFrame was created successfully
mars_temp_df.head()

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
mars_temp_df.dtypes

In [ ]:
# Change data types for data analysis
mars_temp_df['terrestrial_date'] = pd.to_datetime(mars_temp_df['terrestrial_date'])
mars_temp_df['sol'] = mars_temp_df['sol'].astype('int64')
mars_temp_df['ls'] = mars_temp_df['ls'].astype('int64')
mars_temp_df['month'] = mars_temp_df['month'].astype('int64')
mars_temp_df['min_temp'] = mars_temp_df['min_temp'].astype('double')
mars_temp_df['pressure'] = mars_temp_df['pressure'].astype('double')

In [ ]:
# Confirm type changes were successful by examining data types again
mars_temp_df.dtypes

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
# Count the occurrences of each unique month and sort by index
months = mars_temp_df['month'].value_counts().sort_index()
# Print the result
months

In [ ]:
# 2. How many Martian days' worth of data are there?
days = mars_temp_df['sol'].nunique()
# Print the result
days

In [ ]:
# 3. What is the average low temperature by month?
avg_min_temp = mars_temp_df.groupby('month')['min_temp'].mean()
# Print the result
avg_min_temp

In [ ]:
# Plot the average temperature by month
plt.figure()
avg_min_temp.plot(kind='bar')
# Set labels
plt.xlabel('Month')
plt.ylabel('Avg Min Temperature (°C)')
# Show plot
plt.tight_layout()
plt.show()

In [ ]:
# Identify the coldest and hottest months in Curiosity's location
# Sort months by average temperature from coldest to hottest
sorted_months = avg_min_temp.sort_values().index

# Plotting the average minimum temperature by sorted months
plt.figure()
avg_min_temp.loc[sorted_months].plot(kind='bar')
# Set labels
plt.xlabel('Month')
plt.ylabel('Avg Min Temperature (°C)')
# Show plot
plt.tight_layout()
plt.show()

In [ ]:
# 4. Average pressure by Martian month
# Average daily atmospheric pressure for each month
avg_pressure = mars_temp_df.groupby('month')['pressure'].mean()
# Print the result
avg_pressure

In [ ]:
# Workout and sort the pressure for the months by lowest to highest
sorted_pressure_months = avg_pressure.sort_values().index

# Plot the average pressure by month by sorted months
plt.figure()
avg_pressure.loc[sorted_pressure_months].plot(kind='bar')
# Set labels
plt.xlabel('Month')
plt.ylabel('Avg Pressure')
# Show plot
plt.tight_layout()
plt.show()

In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?
# Plotting the daily minimum temperatures
plt.figure()
plt.plot(mars_temp_df['id'], mars_temp_df['min_temp'])
# Set labels and x-axis ticks
plt.xlabel('Number of Terrestrial Days')
plt.ylabel('Minimum Temperature (°C)')
plt.xticks(mars_temp_df['id'].index[::250])
# Show plot
plt.tight_layout()
plt.show()

On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
# Define the filepath
filepath = 'Results/mars_data.csv'
# Export the DataFrame to a CSV file
mars_temp_df.to_csv(filepath, index=False)
# Report file has been exported
print(f"Results exported to '{filepath}'")

In [ ]:
browser.quit()